In [1]:
#dbutils.library.installPyPI("mlflow", extras="extras")
#dbutils.library.installPyPI("azureml-sdk[databricks]")
#dbutils.library.restartPython()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-984644253577107> in <module> 
 1 #dbutils.library.installPyPI("mlflow", extras="extras") 
 ----> 2 dbutils . library . installPyPI ( "azureml-sdk[databricks]" ) 
 3 dbutils . library . restartPython ( ) 

 /local_disk0/tmp/1604348004575-0/dbutils.py in installPyPI (self, project, version, repo, extras) 
 243 def installPyPI ( self , project , version = "" , repo = "" , extras = "" ) : 
 244 return self.print_and_return(self.entry_point.getSharedDriverContext() \
 --> 245 .addIsolatedPyPILibrary(project, version, repo, extras))
 246 
 247 def restartPython ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o990.addIsolatedPyPILibrary.
: org.apache.spark.SparkException: Library utilities are not available on Databricks Runtime for Machine Learning, use %pip/%conda instead. See https://docs.microsoft.com/azure/databricks/libraries/notebooks-python-libraries.
	at com.databricks.backend.daemon.driver.SharedDriverContext.getIsolationId(SharedDriverContext.scala:396)
	at com.databricks.backend.daemon.driver.SharedDriverContext.addIsolatedPyPILibrary(SharedDriverContext.scala:700)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

### Track experiment runs and deploy ML models with MLflow and Azure Machine Learning (preview)

https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-mlflow

In [3]:
user_name = "memasanz"

In [4]:
import mlflow
from azureml.core import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

Connect to Azure ML Workspace - you will be asked to log in

In [6]:
config = {
    "subscription_id": "c778c451-6f3b-48e1-86b1-e55fa7615154",
    "resource_group": "adb-workspace-dev-rg",
    "workspace_name": "adb-aml-workspace"
}

In [7]:
#https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.authentication.interactiveloginauthentication?view=azure-ml-py
#https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.ipynb
  

interactive_auth = InteractiveLoginAuthentication(tenant_id="150e302b-984e-491f-a34e-758d0390a174", cloud="AzureUSGovernment")

ws = Workspace.get(name=config['workspace_name'],
                   subscription_id=config['subscription_id'],
                   resource_group= config['resource_group'], auth=interactive_auth)

Create an Azure ML Experiment

In [9]:
from azureml.core.experiment import Experiment
 
mlflow.set_experiment(experimentName) 

experimentName = user_name + "BaseExperimentWithAzureMLIntegration" 
mlflow.set_experiment(experimentName) 
experiment = Experiment(ws, experimentName)

MLflow Tracking with Azure Machine Learning lets you store the logged metrics and artifacts from your local runs into your Azure Machine Learning workspace.

**Note:** The tracking URI is valid up to an hour or less. If you restart your script after some idle time, use the get_mlflow_tracking_uri API to get a new URI.

In [11]:
print(ws.get_mlflow_tracking_uri())
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

azureml://usgovvirginia.api.ml.azure.us/mlflow/v1.0/subscriptions/c778c451-6f3b-48e1-86b1-e55fa7615154/resourceGroups/adb-workspace-dev-rg/providers/Microsoft.MachineLearningServices/workspaces/adb-aml-workspace?

- Set the MLflow experiment name with set_experiment() and start your training run with start_run(). Then use log_metric() to activate the MLflow logging API and begin logging your training run metrics.

- Any run with MLflow Tracking code in it will have metrics logged automatically to the workspace.

In [14]:
import numpy as np
from sklearn.linear_model import LogisticRegression

import mlflow
import mlflow.sklearn

if __name__ == "__main__":
    with mlflow.start_run():
      X = np.array([-2, -1, 0, 1, 2, 1]).reshape(-1, 1)
      y = np.array([0, 0, 1, 1, 1, 0])
      lr = LogisticRegression()
      lr.fit(X, y)
      score = lr.score(X, y)
      print("Score: %s" % score)
      mlflow.log_metric("score", score)
      mlflow.sklearn.log_model(lr, "model2")
      print("Model saved in run %s" % mlflow.active_run().info.run_uuid)
      mlflow.end_run()

Score: 0.6666666666666666
2020/11/03 03:01:30 WARNING mlflow.models.model: Logging model metadata to the tracking server has failed, possibly due older server version. The model artifacts have been logged successfully under azureml://experiments/memasanzBaseExperimentWithAzureMLIntegration/runs/d9e22962-2ccd-456c-bb01-6cc0586b5ff0/artifacts. In addition to exporting model artifacts, MLflow clients 1.7.0 and above attempt to record model metadata to the tracking store. If logging to a mlflow server via REST, consider upgrading the server version to MLflow 1.7.0 or above.
Model saved in run d9e22962-2ccd-456c-bb01-6cc0586b5ff0